In [1]:
import pandas as pd
import sklearn.datasets
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from itertools import chain, combinations, permutations
import numpy as np

In [2]:
df = pd.read_csv('../../Github/ozone/ozone_model_times.csv')

In [3]:
df

,begin_sha,end_sha,commits,additions,deletions,changed_files,total_time,begin_Dispensables,begin_Bloaters,begin_Change Preventers,begin_Couplers,begin_Object-Orientation Abusers,end_Dispensables,end_Bloaters,end_Change Preventers,end_Couplers,end_Object-Orientation Abusers
0,e950ecee512fd87a9c20c233ba6ed224acd2c6e4,a01251c8113aee036fd7f229612a33c33cb5e558,1,6,3,1,24978000000000,2006,1130.0,1130.0,734.0,734.0,1003.0,566.0,566.0,367.0,910.0
1,6035a8b7df6b622eb5bebe7814f81e349bbe7eb3,534160f96c5dae071192ddb809261ec7905ddbcd,4,60,0,6,27174000000000,1002,565.0,565.0,367.0,367.0,990.0,563.0,563.0,365.0,891.0
2,b9496f9eaf0b9239fcd8b0084d859b4501ccc192,9b59ade990a87a5b8034dcc18be1e997b4f59578,3,72,3,2,43727000000000,1001,565.0,565.0,365.0,365.0,983.0,568.0,568.0,366.0,900.0
3,e950ecee512fd87a9c20c233ba6ed224acd2c6e4,152cbea1bce867709afb498d55cc4f609aa3f98d,3,393,107,7,116274000000000,2006,1130.0,1130.0,734.0,734.0,998.0,565.0,565.0,365.0,907.0
4,55de3fb7e9ca2ba9c2f7b854aa09fcc3c2224532,a6ed346744c69d19c1000cb0c3ad75d9dd4fe669,38,132,23,7,193769000000000,996,563.0,563.0,365.0,365.0,990.0,563.0,563.0,365.0,891.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,d6cbb15f7a29221fcbd8f2c3bc9f5073c12c12d7,c04c01d85452e969ea81d1f8486271e85bff72a9,3,3,2,1,9439818000000000,734,428.0,428.0,241.0,241.0,0.0,0.0,0.0,0.0,0.0
1801,fe7f5b4ecad81dfff7d7703b69e2eec1907a4a95,e01b47184818365dcb7baf528fb2d874209b6b47,7,428,47,8,5353469000000000,1462,854.0,854.0,482.0,482.0,0.0,0.0,0.0,0.0,0.0
1802,af4730a54bd2bbc63d556936292ae760701b7e7f,3eb7235ca879498c2d8ebcd5fd228d6e4cb16891,20,39,35,21,10442829000000000,727,427.0,427.0,233.0,233.0,0.0,0.0,0.0,0.0,0.0
1803,334c6ee65bdc41d0150ad381bbed024a82ea0081,3b89d4daac8747652ee360bac9a0ba1556f12c4f,7,69,0,1,29549932000000000,886,506.0,506.0,298.0,298.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X = df.loc[:, ~df.columns.isin(['begin_sha', 'end_sha', 'begin_time', 'end_time', 'total_time' , 'timedelta' , 'formatted_duration'])]

In [5]:
def scale_data(data,columns):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[columns])

    return pd.DataFrame(scaled_data)

In [6]:
def get_optimal_clusters(data):
    scores = []
    for n_clusters in range(2,10):
        km = KMeans(n_clusters = n_clusters)
        km.fit(data)
        sil_avg = silhouette_score(data, km.labels_).round(4)
        scores.append([sil_avg , n_clusters])
    # scores.sort(reverse=True)
    # return scores[:5]
    return scores

In [7]:
def get_cluster_info(cluster_data):
    cluster_info = []
    # Replace with your time column name
    time_column = 'total_time'

    for cluster_num in cluster_data['cluster'].unique():
        cluster_subset = cluster_data[cluster_data['cluster'] == cluster_num]
        time_min = cluster_subset[time_column].min()
        time_max = cluster_subset[time_column].max()
        time_std = cluster_subset[time_column].std()
        time_minus_2std = time_min - 2 * time_std
        time_minus_1std = time_min - time_std
        time_plus_1std = time_max + time_std
        time_plus_2std = time_max + 2 * time_std
        time_avg = cluster_subset[(cluster_subset[time_column] >= time_minus_1std) & (cluster_subset[time_column] <= time_plus_1std)][time_column].mean()

        cluster_info.append({
            'cluster_num': cluster_num,
            'time_min': time_min,
            'time_max': time_max,
            'time_minus_2std': time_minus_2std,
            'time_minus_1std': time_minus_1std,
            'time_avg': time_avg,
            'time_plus_1std': time_plus_1std,
            'time_plus_2std': time_plus_2std
        })

    return cluster_info

In [12]:
from itertools import combinations
import pandas as pd

# Step 1: Loop through combinations
def column_combinations(data_frame):
    columns = data_frame.columns
    col_combinations = []

    for r in range(1, len(columns) + 1):
        for combo in combinations(columns, r):
            col_combinations.append(combo)

    return col_combinations

# Main function
def main():
    data = X
    combinations_list = column_combinations(data)
    print('Combinations_list ::::' , combinations_list)

    results = []
    for combination in combinations_list:

        print('DATA ::', combination)
        # print('=====================')

        scaled_data = scale_data(data , data.columns)
        # print('SCALED ::', scaled_data)
        # print('====================')

        num_clusters = get_optimal_clusters(scaled_data)
        print('NUM ::', num_clusters)
        print('====================')

        num_clusters.sort()
        num_clusters = num_clusters[::-1]
        num_clusters = num_clusters[:3]

        kmeans = KMeans(n_clusters= max(num_clusters)[0])
        cluster_labels = kmeans.fit_predict(combinations_list)
        df['cluster'] = cluster_labels

        clustered_data = df
        # print("CLUSTERRED DATA ::", clustered_data)
        # print('====================')

        cluster_info = get_cluster_info(clustered_data)
        # print("CL cluster_info ::" , cluster_info)
        # print('====================')


        for info in cluster_info:
            result_dict = {
                'combination': combination,
                'cluster_best': max(num_clusters[0]),
                'cluster_top_3' : num_clusters,
                # 'cluster_num': info['cluster_num'],
                # 'silhouette_score': get_optimal_clusters(scaled_data),
                # 'cluster_sort': num_clusters,
                'time_min': info['time_min'],
                'time_max': info['time_max'],
                'time_minus_2': info['time_minus_2std'],
                'time_minus_1': info['time_minus_1std'],
                'time_avg': info['time_avg'],
                'time_plus_1': info['time_plus_1std'],
                'time_plus_2': info['time_plus_2std']
            }
            results.append(result_dict)

    results_df = pd.DataFrame(results)
    results_df.to_csv('output_ozne.csv', index=False)  # Save results to CSV

if __name__ == "__main__":
    main()

Combinations_list :::: [('commits',), ('additions',), ('deletions',), ('changed_files',), ('begin_Dispensables',), ('begin_Bloaters',), ('begin_Change Preventers',), ('begin_Couplers',), ('begin_Object-Orientation Abusers',), ('end_Dispensables',), ('end_Bloaters',), ('end_Change Preventers',), ('end_Couplers',), ('end_Object-Orientation Abusers',), ('commits', 'additions'), ('commits', 'deletions'), ('commits', 'changed_files'), ('commits', 'begin_Dispensables'), ('commits', 'begin_Bloaters'), ('commits', 'begin_Change Preventers'), ('commits', 'begin_Couplers'), ('commits', 'begin_Object-Orientation Abusers'), ('commits', 'end_Dispensables'), ('commits', 'end_Bloaters'), ('commits', 'end_Change Preventers'), ('commits', 'end_Couplers'), ('commits', 'end_Object-Orientation Abusers'), ('additions', 'deletions'), ('additions', 'changed_files'), ('additions', 'begin_Dispensables'), ('additions', 'begin_Bloaters'), ('additions', 'begin_Change Preventers'), ('additions', 'begin_Couplers'),

C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\patcharaprapa_k\AppData\Local\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly t

NUM :: [[0.5939, 2], [0.7352, 3], [0.774, 4], [0.8046, 5], [0.8079, 6], [0.7389, 7], [0.6961, 8], [0.6393, 9]]


InvalidParameterError: The 'n_clusters' parameter of KMeans must be an int in the range [1, inf). Got 0.8079 instead.

In [11]:
for combination in combinations_list:

        print('DATA ::', combination)

NameError: name 'combinations_list' is not defined